<a href="https://colab.research.google.com/github/AlexLefte/POS-Tagging-Using-Hidden-Markov-Models/blob/main/HMM_Project_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTUL MODULELOR NECESARE**

---



In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import time

# **HMM MODEL**

---



In [ ]:
class HMM(object):
  def __init__(self):
    self.initial_prob = []    # Vector de probabilități inițiale
    self.trans_prob = []      # Matrice de tranziție
    self.emm_prob = []        # Matrice de emisie
    self.hidden_states = []   # Lista stărilor ascunse (unice)
    self.M = 0                # Număr de stări ascunse
    self.observations = []    # Lista observațiilor (unice)
    self.tagged_data = []     # Tupluri (Cuvânt/Stare) de antrenare
    self.rules = None         # Pattern-uri regex
    return


  # Metoda pentru determinarea vectorului de probabilitati initiale pe setul 
  # de date destinat antrenarii
  def set_initial_prob(self, initial_tags):
    self.initial_prob = []
    sentences = len(initial_tags)
    initial_tags = [tag for name, tag in initial_tags] 

    # Pentru fiecare stare ascunsă din vocabular => calculează probabilitatea inițială
    for state in self.hidden_states:
      self.initial_prob.append(initial_tags.count(state) / sentences)
    return
    

  # Metoda pentru probabilității de tranziție de la starea ascunsă ai -> aj:
  def aij(self, ai, aj, tags):
    # Determinare număr total de apariții ale stării ai 
    count_ai = tags.count(ai)
    # Inițializare număr tranziții aj -> ai
    count_aj_ai = 0

    # Determinare perechi consecutive ale celor două stări ascunse, iterând
    # prin stările setului de antrenare.
    for index in range(len(tags)-1):
        if tags[index] == ai and tags[index + 1] == aj:
            count_aj_ai += 1

    # Întoarce tuplul format din celi doi contori
    return (count_aj_ai, count_ai)


  # Metodă pentru determinarea matricei de tranziții
  def compute_trans_matrix(self, tags):
    # Inițializare matrice cu dimensiunea M x M (unde M - numar stari ascunse)
    tags_matrix = np.zeros((self.M, self.M), dtype='float32')
    
    # Iterează prin lista de stări ascunse și calculează matricea de tranziții
    for i, state_i in enumerate(self.hidden_states):
      for j, state_j in enumerate(self.hidden_states): 
        tags_matrix[i, j] = self.aij(state_i, state_j, tags)[0] / self.aij(state_i, state_j, tags)[1]
    return tags_matrix


  # Metoda pentru probabilității de emisie ai -> bi:
  def ai_bi(self, ai, bi, data):
    # Extragere stări ascunse din fiecare tuplu (obs, tag)
    states = [tpl[1] for sentence in data for tpl in sentence]

    # Determinare număr total de apariții ale stării ai
    count_ai = states.count(ai) 
    
    # Determinare perechi stare (ai) -> simbol (bi)
    count_ai_bi = 0
    for sentence in data:
      for word, tag in sentence:
        if word == bi and tag == ai:
            count_ai_bi += 1
    return (count_ai_bi, count_ai)


  # Metodă pentru determinarea matricei de emisie
  def compute_emm_matrix(self, train_data):
    # Inițializare matrice cu dimensiunea M x v (unde M - numar stari ascunse și
    # v - număr total observații
    emm_matrix = np.zeros((self.M, len(self.observations)), dtype='float32')

    # Iterează prin lista de stări ascunse și calculează matricea de emisie
    for i, state_i in enumerate(self.hidden_states):
      for j, obs_j in enumerate(self.observations): 
        emm_matrix[i, j] = self.ai_bi(state_i, obs_j, train_data)[0] / self.ai_bi(state_i, obs_j, train_data)[1]
    return emm_matrix


  # Metoda forward:
  def forward(self, V):
    # V -> secvență de observații

    # Inițializare matrice alpha
    alpha = np.zeros((len(V), self.M))

    # Determinare index observație inițială
    index = self.observations.index(V[0])

    # Determinarea primei linii din matricea alpha = 
    # = π * [ coloana dn matricea B corespunzătoare primei observații]
    alpha[0, :] = self.initial_prob * self.emm_prob[:, index]

    # Se iterează prin restul colecției și se determină următoarele linii
    for t in range(1, len(V)):
      for j in range(self.M):
        # Se determină indexul corespunzător observației următoare
        col_index = self.observations.index(V[t])
        # Se calculează: alphaj(t + 1) = sum(aij * alphai(t)) * bjkv(t+1) 
        alpha[t, j] = alpha[t - 1].dot(self.trans_prob[:, j]) * self.emm_prob[j, col_index]
    
    # Întoarce probabilitatea de a observa vectorul V, respectiv matricea alpha
    return np.sum(alpha[-1, :]), alpha


  # Metoda Viterbi:
  def Viterbi(self, observations, train_data):
    # Inițializarea listei de stări ascunse
    states = []

    # Se iterează prin lista de observații vizualizate (V = observations)
    for index, word in enumerate(observations):  
      # Inițializare listă de probabilități la momentul t = index
      probs = []

      # Dacă observația nu a fost întîlnită în setul de antrenare, se va încerca
      # asignarea unei stări utilizând expresii regulate
      if word not in self.observations:
        state = self.rules.tag([word])[0][1]
      else:
        # Observația se află în listă => determinăm indexul
        column = self.observations.index(word)
        # Dacă ne aflăm la prima iterație, vom folosi matricea probabilităților
        # inițiale.
        if index == 0:  
          for i in range(self.M):
            # Adaugă fiecare probabilitate: alphai(1) = πi * bikv(1)
            probs.append(self.initial_prob[i] * self.emm_prob[i, column])
        else:
          # Determinarea indexului ultimei stări prezise.
          last_state = self.hidden_states.index(states[-1])
          for i in range(self.M):
            # Adaugă fiecare probabilitate: alphaj(index) = aij * bjkv(index)
            probs.append(self.trans_prob[last_state, i] * self.emm_prob[i, column])
        # Determinarea stării corespunzătoare celei mai mari probabilități
        state = self.hidden_states[probs.index(max(probs))] 
      # Adăugarea stării
      states.append(state)    

    # Se întoarce lista de predicții
    return states
          

  # Metoda pentru antrenarea modelului:
  def train(self, train_data):
    start = time.time()
    # Salvare date de train
    self.tagged_data = [ tup for sent in train_data for tup in sent ]

    # Initializarea listei de stari ascunse
    self.hidden_states = list({state for word, state in self.tagged_data})
    self.M = len(self.hidden_states)
    self.observations = list({word for word, state in self.tagged_data})

    # Determinarea vectorului de probabilitati initiale:
    self.set_initial_prob([sentence[0] for sentence in train_data])

    # Determinarea matricelor de transmisie/emisie:
    self.trans_prob = self.compute_trans_matrix([state for word, state in self.tagged_data])
    self.emm_prob = self.compute_emm_matrix(train_data)

    # Crearea unor pattern-uri regex pentru a îmbunătății perfomanța
    patterns = [
    (r'.*ing$', 'VERB'),              # gerunziu
    (r'.*ed$', 'VERB'),               # verb la timpul trecut 
    (r'.*es$', 'VERB'),               # verb la present simplu (persoana a 3 a)
    (r'.*\'s$', 'NOUN'),              # posesiv
    (r'.*s$', 'NOUN'),                # substantive la plural
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # numere
    (r'.*', 'NOUN')  
    ]

    # Clasă utilizată pentru atribuirea tag-urilor (prin comparația cu patternul definit)
    self.rules = nltk.RegexpTagger(patterns)

    # Afișare timp necesar antrenării
    end = time.time()
    print("Time taken for training (in minutes): " + str((end - start) / 60))
    return


  # Metoda de test:
  def test(self, test_data, train_data):
    # Identificare observații din setul de test
    test_data_words = [word for sentence in test_data for word, state in sentence]

    # Identificare secvență de stări din setul de test 
    test_data_states = [state for sentence in test_data for word, state in sentence]
    
    # Începe algoritmul Viterbi
    start = time.time()
    pred = self.Viterbi(test_data_words, train_data)
    end = time.time()

    # Afisarea timpului necesar testarii:
    print("Time taken for testing (in minutes): " + str((end - start) / 60)) 

    # Determinarea performanței modelului (acuratețe):
    cnt = 0
    for i in range(len(pred)):
      if pred[i] == test_data_states[i]:
        cnt += 1

    print('Accuracy: ', str(cnt / len(pred) * 100), "%")
    print("Real states: ", test_data_states)
    print("Predicted states: ", pred)

    return

# **OBȚINEREA SETULUI DE DATE**

---



In [ ]:
# Download 'Native language toolkit universal tagset' 
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
# Salvarea datelor
data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [ ]:
# Vizualizarea unor fraze prezente în setul nostru de date:
i = 0
for sentence in data[:5]:
  i += 1
  print(f"Sentence {i}:")
  for word in sentence:
    print(word[0], end=' ')
  print('\n\n')

Sentence 1:
Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 . 


Sentence 2:
Mr. Vinken is chairman of Elsevier N.V. , the Dutch publishing group . 


Sentence 3:
Rudolph Agnew , 55 years old and former chairman of Consolidated Gold Fields PLC , was named *-1 a nonexecutive director of this British industrial conglomerate . 


Sentence 4:
A form of asbestos once used * * to make Kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed * to it more than 30 years ago , researchers reported 0 *T*-1 . 


Sentence 5:
The asbestos fiber , crocidolite , is unusually resilient once it enters the lungs , with even brief exposures to it causing symptoms that *T*-1 show up decades later , researchers said 0 *T*-2 . 




# **SETURI TRAIN/TEST**
---



In [ ]:
# Împărțirea setului în proporție: train - 80%, test - 20%
train_set, test_set = train_test_split(data, test_size=0.2, train_size=0.8, shuffle=True)

In [ ]:
# Afișarea numărului de cuvinte corespunzătoare fiecărui set:
train_words = [ tup for sent in train_set for tup in sent ]
test_words = [ tup for sent in test_set for tup in sent ]
print('Number of tagged words inside the train set: ' + str(len(train_words)))
print('Number of tagged words inside the test set: ' + str(len(test_words)))

# Identificarea stărilor unice:
pos_tags = {tag for word,tag in train_words}
print('\nNumber of unique POS tags: ' + str(len(pos_tags)))
print('List of POS tags: ')
print(pos_tags)

Number of tagged words inside the train set: 81103
Number of tagged words inside the test set: 19573

Number of unique POS tags: 12
List of POS tags: 
{'NOUN', 'PRON', 'DET', '.', 'VERB', 'PRT', 'CONJ', 'ADJ', 'ADV', 'X', 'NUM', 'ADP'}


# **ANTRENAREA MODELULUI**

---



In [ ]:
# Creare model
hmm = HMM()

In [ ]:
# Antrenare
hmm.train(train_set[:10000])

Time taken for training (in minutes): 101.97938428322475


# **Vizualizarea parametrilor modelului**

---



In [ ]:
print("Stări ascunse: ", hmm.hidden_states, "\n")
print("Matricea de tranziție: \n", hmm.trans_prob, "\n")
print("Matricea de emisie: \n", hmm.emm_prob, "\n")
print(f'Vector de probabilități inițiale: {hmm.initial_prob}')
print('Suma probabilităților inițiale: ', end="")
print(str(sum(hmm.initial_prob)) + "\n")

Stări ascunse:  ['NOUN', 'PRON', 'DET', '.', 'VERB', 'PRT', 'CONJ', 'ADJ', 'ADV', 'X', 'NUM', 'ADP'] 

Matricea de tranziție: 
 [[2.67195195e-01 4.71394882e-03 1.25562456e-02 2.39211485e-01
  1.46432400e-01 4.37111631e-02 4.26826663e-02 1.22562675e-02
  1.70559250e-02 2.87979431e-02 9.85643920e-03 1.75530314e-01]
 [2.12708429e-01 8.11176188e-03 9.46372282e-03 3.92068513e-02
  4.85804409e-01 1.26182968e-02 4.95718792e-03 7.16538951e-02
  3.33483554e-02 9.01306868e-02 9.01306886e-03 2.29833256e-02]
 [6.43041968e-01 3.30649805e-03 5.31914877e-03 1.82576198e-02
  3.86716500e-02 1.43760786e-04 4.31282358e-04 2.00833812e-01
  1.32259922e-02 4.57159281e-02 2.18516383e-02 9.20069031e-03]
 [2.22883880e-01 6.62484020e-02 1.73862189e-01 9.28328335e-02
  8.97490457e-02 1.80774136e-03 5.67843467e-02 4.60442379e-02
  5.27435131e-02 2.56273933e-02 7.98596367e-02 9.14504454e-02]
 [1.07424378e-01 3.52887250e-02 1.32172316e-01 3.40971574e-02
  1.68469295e-01 3.05224564e-02 5.31622348e-03 6.68194294e-02


In [ ]:
# Realizarea unui tabel pentru observarea matricelor de tranzitie/emisie.
transition_table = pd.DataFrame(hmm.trans_prob, columns = list(hmm.hidden_states), index = list(hmm.hidden_states))
display(transition_table)
print()

emmision_table = pd.DataFrame(hmm.emm_prob, columns = list(hmm.observations), index = list(hmm.hidden_states))
display(emmision_table)

,NOUN,PRON,DET,.,VERB,PRT,CONJ,ADJ,ADV,X,NUM,ADP
NOUN,0.267195,0.004714,0.012556,0.239211,0.146432,0.043711,0.042683,0.012256,0.017056,0.028798,0.009856,0.175530
PRON,0.212708,0.008112,0.009464,0.039207,0.485804,0.012618,0.004957,0.071654,0.033348,0.090131,0.009013,0.022983
DET,0.643042,0.003306,0.005319,0.018258,0.038672,0.000144,0.000431,0.200834,0.013226,0.045716,0.021852,0.009201
.,0.222884,0.066248,0.173862,0.092833,0.089749,0.001808,0.056784,0.046044,0.052744,0.025627,0.079860,0.091450
VERB,0.107424,0.035289,0.132172,0.034097,0.168469,0.030522,0.005316,0.066819,0.081852,0.221082,0.022456,0.094500
PRT,0.251254,0.018912,0.094558,0.039367,0.402161,0.001544,0.002702,0.087997,0.010807,0.013894,0.057121,0.019684
CONJ,0.349589,0.060822,0.115616,0.033973,0.158356,0.004932,0.000548,0.119452,0.050959,0.009863,0.041644,0.054247
ADJ,0.699884,0.000775,0.005041,0.064172,0.011051,0.011051,0.017449,0.067080,0.003877,0.021908,0.019581,0.078131
ADV,0.030540,0.014096,0.071652,0.135865,0.350039,0.013704,0.006656,0.127251,0.076351,0.023884,0.030540,0.119421
X,0.062325,0.056710,0.053154,0.164327,0.199701,0.185663,0.011230,0.017406,0.027887,0.074864,0.001684,0.145050


,sell-offs,Biondi-Santi,1961,Embassy,outstanding,learning,retained,adopt,government-owned,Valley,...,Stockholders,Instead,good-natured,Telegraph,five-day,vehicle,himself,track,interim,declaring
NOUN,0.000043,0.000043,0.000000,0.000129,0.000000,0.000043,0.000000,0.000000,0.000000,0.000471,...,0.000043,0.000000,0.000000,0.000171,0.000000,0.000129,0.000000,0.000043,0.000000,0.000000
PRON,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002253,0.000000,0.000000,0.000000
DET,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VERB,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000183,0.000092,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000275
PRT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CONJ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ADJ,0.000000,0.000000,0.000000,0.000000,0.003296,0.000000,0.000000,0.000000,0.000194,0.000000,...,0.000000,0.000000,0.000194,0.000000,0.000194,0.000000,0.000000,0.000000,0.000388,0.000000
ADV,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.001958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
X,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# **Testarea modelului** 
---



In [ ]:
# Testarea pe un lot mai limitat de date de test
hmm.test(test_set[:10], train_set)

Time taken for testing (in minutes): 0.0013930797576904297
Accuracy:  96.69421487603306 %
Real states:  ['NOUN', 'PRT', 'DET', 'NOUN', 'VERB', 'VERB', 'ADP', 'PRON', 'VERB', 'VERB', 'DET', 'NUM', 'NUM', 'NOUN', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'DET', 'NUM', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NUM', '.', 'VERB', 'VERB', 'VERB', 'X', 'ADP', 'NUM', 'NUM', 'NOUN', '.', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN', 'VERB', 'X', 'PRT', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', '.', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', '.', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'VERB', 'ADJ', '.', 'ADV', 'PRON', 'VERB', 'ADV', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', '.', 'ADP', 'PRON', 'VERB', 'DET', 'NOUN', '.', 'PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'NOUN', '.', 'DET', 'NOUN', 'PRT', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', '.', 'NUM', 'NUM', 'X', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'PRON', 'VERB', 'ADP', 'PRON', 'NOUN', 'ADP', 

In [ ]:
# Testarea pe întregul lot rămas disponibil pentru testare:
hmm.test(test_set[10:], train_set)

Time taken for testing (in minutes): 0.1025715986887614
Accuracy:  95.04940251409653 %
Real states:  ['ADJ', 'NOUN', '.', 'NOUN', 'NUM', '.', 'NUM', '.', 'ADJ', 'NOUN', '.', '.', 'NUM', 'X', 'DET', 'NOUN', 'CONJ', 'NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'DET', 'NOUN', '.', 'CONJ', 'DET', 'NOUN', 'VERB', 'ADP', 'DET', 'ADP', 'PRON', 'VERB', 'VERB', 'ADJ', 'NOUN', '.', 'PRON', 'VERB', 'VERB', 'ADP', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADJ', 'ADP', 'PRON', 'VERB', 'VERB', 'NOUN', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'CONJ', 'ADP', 'PRON', '.', 'ADJ', '.', 'NOUN', '.', 'NOUN', '.', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'VERB', 'ADV', '.', 'DET', 'NOUN', 'ADV', 'VERB', 'X', 'PRON', 'VERB', 'VERB', 'DET', 'NOUN', 'CONJ', 'VERB', 'PRT', 'NUM', 'NOUN', 'ADP', 'PRON', 'NOUN', 'NOUN', '.', 'ADV', 'X', 'VERB', 'NOUN', 'PRT', 'ADV', 'ADP', 'NUM', 'ADP', 'ADP', 'NUM', '.', 'NOUN', 'NOUN', 'NOUN', '.', 'DET', 'NOUN', 'ADJ', 'NOUN', '.', 'ADV', 'VERB', 'X', 'NOUN', 'NOUN', 'VERB', 'PRON', 'PRT', 'VER

In [ ]:
# Testarea metodei forward pe o propoziție scrisă aleator:
print(hmm.forward(["Rudolph", "said", "Kent", "cigarettes", "are", "recently", "diagnosed", "with", "cancer", "."])[0])

3.710526622952651e-34


In [ ]:
# Testarea metodei Viterbi pe fraze scrise aleator:
print(hmm.Viterbi(["Rudolph", "Agnew", "said", "Kent", "cigarettes", "are", "recently", "diagnosed", "with", "cancer", "."], train_set))
print(hmm.Viterbi(["Jane", "will", "spot", "Will", "."], train_set))
print(hmm.Viterbi(["Will", "saw", "Mary", "."], train_set))
print(hmm.Viterbi(["Spot", "likes", "Jane", "."], train_set))

['NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'ADV', 'VERB', 'ADP', 'NOUN', '.']
['NOUN', 'VERB', 'NOUN', 'NOUN', '.']
['NOUN', 'VERB', 'NOUN', '.']
['NOUN', 'VERB', 'NOUN', '.']
